In [16]:
from pathlib import Path

import dash
import dash_core_components as dcc
from dash.dependencies import Input, Output
import dash_html_components as html
from jupyter_dash import JupyterDash
from matplotlib.cm import get_cmap
from matplotlib.colors import to_hex
import numpy as np
import pandas as pd
import plotly.graph_objects as go

### Load the data

In [17]:
DATA_ROOT = Path('..') / 'data'

In [18]:
dfs = []
activity_labels = ['bed', 'chair', 'lying', 'ambulating']
default_names = ['time', 'front', 'vertical', 'lateral', 'sensor_id', 'rssi', 'phase', 'frequency', 'activity']
for data_file in Path(DATA_ROOT).rglob('d[12]p??[FM]'):
    df = pd.read_csv(data_file, names=default_names)
    df['activity_label'] = df['activity'].apply(lambda i: activity_labels[i - 1])
    df['gender_label'] = str(data_file)[-1]
    df['participant'] = data_file.name
    
    # Add a column indicating order of the activities for a particiapnt.
    df = df.sort_values(by=['time'])
    df['activity_sequence'] = (df['activity'].shift(1) != df['activity']).cumsum()
    dfs.append(df)

sensor_df = pd.concat(dfs, axis='index')
sensor_df = sensor_df.sort_values(by=['participant', 'time'])

sensor_df.to_csv('sensor_df.csv')

In [19]:
# Set up some colors for the sensors and the activities.

colors = get_cmap('tab10').colors
values = ['front', 'lateral', 'vertical']
SENSOR_COLORS = {
    value: to_hex(colors[i]) for i, value in enumerate(values)}

colors = get_cmap('Pastel1').colors
values = (sensor_df[['activity', 'activity_label']]
          .drop_duplicates()
          .to_dict(orient='records'))
ACTIVITY_COLORS = {}
for i, value in enumerate(values):
    ACTIVITY_COLORS[value['activity']] = ACTIVITY_COLORS[value['activity_label']] = to_hex(colors[i])


### Plot the data

In [31]:
app = JupyterDash(__name__)
# Define the layout of the app.
# Requires a bit of HTML and CSS knowledge - but but _too_ much :)
app.layout = html.Div(children=[
    html.Div(children=[
        dcc.Graph(id='plot'),
    ]),
    html.Div(children=[
            html.Button(
                '⇦', 
                id='prev-participant', 
                n_clicks=0,
                style={'fontSize': '200%'}),
            html.Button(
                '⇨', 
                id='next-participant', 
                n_clicks=0,
                style={'fontSize': '200%'}),
        ], style={'display': 'flex', 'justifyContent': 'space-between'}),
    html.Div(children=[
        html.Div(children=[
            html.Label('Smoothing'),
            html.Div(children=[
                dcc.Slider(
                    id='smoothing-slider', 
                    min=0, 
                    max=10, 
                    value=0, 
                    marks={str(m):'' for m in [0, 10]}, step=1)
            ], style=dict(minWidth='256px', maxHeight='10px')),
            html.Label('Kernel', style={'padding-right': '0.5rem'}),
            dcc.Dropdown(
                id='kernel-dropdown',
                options=[
                    {'label': 'Rectangle', 'value': 'rectangle'},
                    {'label': 'Hamming', 'value': 'hamming'},
                ],
                value='rectangle', 
                multi=False,
                clearable=False,
                style={'fontFamily': 'sans-serif', 'minWidth': 192}),
        ], style=dict(display='flex', flexDirection='row', alignItems='center', paddingLeft='1rem'))
    ],
    style=dict(display='flex', justifyContent='center', width='100%', alignItems='center'))
], style={'fontFamily': 'sans-serif'})

# The callback responds to changes in its inputs
# (i.e., the buttons and slider) and outputs a new figure
# on the basis of these values.
@app.callback(
    Output('plot', 'figure'),
    [
        Input('next-participant', 'n_clicks'), 
        Input('prev-participant', 'n_clicks'),
        Input('smoothing-slider', 'value'),
        Input('kernel-dropdown', 'value')]
)
def update_figure(next_button_clicks, prev_button_clicks, smoothing, kernel_name):
    """When something changes (e.g., button click or smoothing change) update the figure"""
   
    participants = sensor_df['participant'].unique()
    
    # Only have the number of time the buttons have been clicked
    # (this is all I understand for now) so mess around to get
    # the index of the participant.
    participant_index = (next_button_clicks - prev_button_clicks) % len(participants)
    while participant_index < 0:
        participant_index += len(participants)
    participant = participants[participant_index]
    
    # Subset the data
    df = sensor_df[sensor_df['participant'] == participant]
    
    xmin, xmax = df['time'].min(), df['time'].max()
    nactivities = df['activity_sequence'].nunique()
    activity_labels = []
    
    if smoothing:
        window = smoothing * 2 + 1
        if kernel_name == 'hamming':
            kernel = np.hamming(window)
        else:
            kernel = np.ones(window)
                    
    # Plot the figure.
    fig = go.Figure()
    for index, group in enumerate(df.groupby('activity_sequence')):
        _, activity_df = group
        start = activity_df['time'].min()
        end = activity_df['time'].max()
        
        # Plot the activity rectangles.
        fig.add_trace(go.Scatter(
            x=[start, end, end, start],
            y=[-2, -2, 2, 2],
            line_color=ACTIVITY_COLORS[activity_df['activity'].values[0]],
            line_width=0,
            fill='toself',
            mode='lines',
            showlegend=False,
            name='',
        ))
        
        # Plot the sensor data.
        for sensor in ['front', 'vertical', 'lateral']:
            original = activity_df[sensor].values
            
            # Smooth if required using moving average.
            if smoothing:
                # To avoid extreme boundary effects, we repeat the first and last values.
                n = len(kernel)
                signal = np.concatenate((
                    np.repeat(original[0], n),
                    original,
                    np.repeat(original[-1], n)))
                smoothed = np.convolve(kernel / np.sum(kernel), signal)[n:-n]
                    
                # Show origin values
                fig.add_trace(go.Scatter(
                    x=activity_df['time'],
                    y=original,
                    line_color=SENSOR_COLORS[sensor],
                    mode='lines',
                    opacity=0.333,
                    showlegend=False,
                    name='',
            ))
            else:
                smoothed = original
            
            # Plot the points
            fig.add_trace(go.Scatter(
                x=activity_df['time'],
                y=smoothed,
                line_color=SENSOR_COLORS[sensor],
                mode='lines+markers',
                marker_size=5,
                showlegend=False,
                name='',
                opacity=1,
            ))
            
        # Need to store the positions and text of the activity labels
        # to ensure they are drawn after everything else otherwise
        # subsequent activity rectangles will be drawn over the labels.
        x = (start + end) / 2
        y = -2 + (index + 0.5) / nactivities * 4
        activity_labels.append((x, y, activity_df['activity_label'].values[0]))
    
    # Done plotting, so draw activity labels.
    for x, y, text in activity_labels:
        fig.add_annotation(
        x=x, 
        y=y, 
        xref='x', # use axis coordinates
        yref='y', # use axis coordinates
        showarrow=False, 
        text=text)
    
    # Show the participant label
    fig.add_annotation(
        x=(xmin + xmax) / 2, 
        y=-2, 
        xref='x', 
        yref='y', 
        yanchor='bottom',
        showarrow=False, 
        text=f'{participant} {participant_index + 1}/{len(participants)}')
    
    # Set the margins 
    fig.update_layout(
        margin=dict(l=24, r=24, t=32, b=32)
    )
    
    # Trick to stop the zoom/pan resetting when data is updated
    fig['layout']['uirevision'] = True
    
    #  Figure is complete, so return it and let Dash to everything else
    return fig

app.run_server(debug=True, mode='inline', port=8051)